# Phase IV: Model Implementation & Fine-Tuning

This notebook implements the complete training pipeline for fine-tuning DistilBERT on the SQuAD v1.1 dataset for question answering.

**Based on Previous Phases:**
- **Phase II EDA**: Optimized parameters (max_length=384, doc_stride=128)
- **Phase III Preprocessing**: Validated tokenization pipeline
- **Model Choice**: DistilBERT for speed/accuracy balance

**Training Strategy:**
- Use Hugging Face Trainer API for efficient training
- Implement proper validation and metrics tracking
- Save checkpoints and monitor training progress
- Optimize hyperparameters based on validation performance

In [1]:
# Phase IV: Model Implementation & Fine-Tuning

# Import libraries
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))

import torch
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from src.preprocessing import prepare_train_features
import matplotlib.pyplot as plt
import json
from tqdm import tqdm

print("Libraries imported successfully!")

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

c:\Users\My Device\Desktop\Question Answering with Transformers_NLP\localenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Libraries imported successfully!
Using device: cpu


## 1. Model Selection & Setup

Load DistilBERT model and tokenizer with optimized configuration from Phase III.

In [2]:
# Model configuration based on Phase III findings
MODEL_NAME = "distilbert-base-uncased"
MAX_LENGTH = 384  # From Phase III validation
DOC_STRIDE = 128  # From Phase III optimization

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)

# Move model to device
model.to(device)

print(f"Model loaded: {MODEL_NAME}")
print(f"Model parameters: {model.num_parameters():,}")
print(f"Model trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
print(f"Max sequence length: {MAX_LENGTH}")
print(f"Document stride: {DOC_STRIDE}")

c:\Users\My Device\Desktop\Question Answering with Transformers_NLP\localenv\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\My Device\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading weights: 100%|██████████| 100/100 [00:00<00:00, 221.44it/

Model loaded: distilbert-base-uncased
Model parameters: 66,364,418
Model trainable parameters: 66,364,418
Max sequence length: 384
Document stride: 128


## 2. Data Loading & Preprocessing

Load SQuAD dataset and apply the validated preprocessing pipeline from Phase III.

In [3]:
# Load datasets
print("Loading SQuAD v1.1 dataset...")
raw_datasets = load_dataset("squad")

print(f"Train samples: {len(raw_datasets['train'])}")
print(f"Validation samples: {len(raw_datasets['validation'])}")

# For initial training, use a subset to verify everything works
TRAIN_SUBSET_SIZE = 1000  # Start with smaller subset for testing
VAL_SUBSET_SIZE = 200    # Smaller validation set

train_dataset = raw_datasets['train'].select(range(TRAIN_SUBSET_SIZE))
val_dataset = raw_datasets['validation'].select(range(VAL_SUBSET_SIZE))

print(f"\nUsing subsets:")
print(f"Train: {len(train_dataset)} samples")
print(f"Validation: {len(val_dataset)} samples")

Loading SQuAD v1.1 dataset...
Train samples: 87599
Validation samples: 10570

Using subsets:
Train: 1000 samples
Validation: 200 samples


In [4]:
# Apply preprocessing function
def preprocess_dataset(dataset, tokenizer, max_length=MAX_LENGTH, doc_stride=DOC_STRIDE):
    """Apply preprocessing to dataset with progress tracking."""
    # Convert to format expected by preprocessing function
    dataset_dict = {
        "question": [example["question"] for example in dataset],
        "context": [example["context"] for example in dataset],
        "answers": [example["answers"] for example in dataset]
    }
    
    # Apply preprocessing
    processed = prepare_train_features(dataset_dict, tokenizer, max_length, doc_stride)
    
    # Convert to torch format
    processed_features = {
        'input_ids': torch.tensor(processed['input_ids']),
        'attention_mask': torch.tensor(processed['attention_mask']),
        'start_positions': torch.tensor(processed['start_positions']),
        'end_positions': torch.tensor(processed['end_positions'])
    }
    
    return processed_features

# Preprocess datasets
print("Preprocessing training data...")
train_processed = preprocess_dataset(train_dataset, tokenizer)
print(f"Training features: {len(train_processed['input_ids'])}")

print("Preprocessing validation data...")
val_processed = preprocess_dataset(val_dataset, tokenizer)
print(f"Validation features: {len(val_processed['input_ids'])}")

print(f"\nPreprocessing complete!")
print(f"Train expansion ratio: {len(train_processed['input_ids'])/len(train_dataset):.2f}x")
print(f"Val expansion ratio: {len(val_processed['input_ids'])/len(val_dataset):.2f}x")

Preprocessing training data...
Training features: 1032
Preprocessing validation data...
Validation features: 200

Preprocessing complete!
Train expansion ratio: 1.03x
Val expansion ratio: 1.00x


## 3. Training Configuration

Configure training arguments based on our dataset characteristics and available resources.

In [5]:
# Training configuration
BATCH_SIZE = 8  # Adjust based on GPU memory
LEARNING_RATE = 3e-5  # Standard for transformer fine-tuning
NUM_EPOCHS = 3  # Start with 3 epochs for initial training
WEIGHT_DECAY = 0.01  # Prevent overfitting
WARMUP_STEPS = 100  # Gradual learning rate warmup

# Calculate training steps
TOTAL_TRAIN_STEPS = (len(train_processed['input_ids']) // BATCH_SIZE) * NUM_EPOCHS

print("=== Training Configuration ===")
print(f"Batch size: {BATCH_SIZE}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Epochs: {NUM_EPOCHS}")
print(f"Weight decay: {WEIGHT_DECAY}")
print(f"Warmup steps: {WARMUP_STEPS}")
print(f"Total training steps: {TOTAL_TRAIN_STEPS}")

# Memory estimate
memory_per_batch = BATCH_SIZE * MAX_LENGTH * 4 * 3  # input_ids, attention_mask, labels
print(f"\nEstimated memory per batch: {memory_per_batch / 1024**2:.1f} MB")
print(f"Recommended GPU memory: {memory_per_batch / 1024**2 * 4:.1f} MB (with gradients)")

=== Training Configuration ===
Batch size: 8
Learning rate: 3e-05
Epochs: 3
Weight decay: 0.01
Warmup steps: 100
Total training steps: 387

Estimated memory per batch: 0.0 MB
Recommended GPU memory: 0.1 MB (with gradients)


In [ ]:
# Setup training arguments
training_args = TrainingArguments(
    output_dir="../models/checkpoints",
    eval_strategy="epoch",  # Fixed: evaluation_strategy -> eval_strategy
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=WEIGHT_DECAY,
    warmup_steps=WARMUP_STEPS,
    logging_dir="../logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    push_to_hub=False,
    report_to=None,  # Disable wandb/tensorboard for now
    dataloader_pin_memory=True,
    fp16=torch.cuda.is_available(),  # Use mixed precision if GPU available
    gradient_checkpointing=True,  # Save memory
    remove_unused_columns=False  # Keep all columns for QA task
)

print("Training arguments configured:")
print(f"  Output directory: {training_args.output_dir}")
print(f"  Evaluation strategy: {training_args.eval_strategy}")
print(f"  Save strategy: {training_args.save_strategy}")
print(f"  Mixed precision: {training_args.fp16}")
print(f"  Gradient checkpointing: {training_args.gradient_checkpointing}")

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

## 4. Data Collator & Trainer Setup

Setup data collator for padding and initialize the Trainer.

In [ ]:
# Create custom dataset class for our processed data
class QADataset(torch.utils.data.Dataset):
    def __init__(self, features):
        self.features = features
    
    def __len__(self):
        return len(self.features['input_ids'])
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.features['input_ids'][idx],
            'attention_mask': self.features['attention_mask'][idx],
            'start_positions': self.features['start_positions'][idx],
            'end_positions': self.features['end_positions'][idx]
        }

# Create dataset objects
train_dataset_obj = QADataset(train_processed)
val_dataset_obj = QADataset(val_processed)

# Data collator for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

print(f"Training dataset size: {len(train_dataset_obj)}")
print(f"Validation dataset size: {len(val_dataset_obj)}")
print(f"Data collator configured with tokenizer: {tokenizer.name_or_path}")

# Test a sample batch
sample_batch = [train_dataset_obj[0], train_dataset_obj[1]]
collated_batch = data_collator(sample_batch)
print(f"\nSample batch shapes:")
for key, value in collated_batch.items():
    print(f"  {key}: {value.shape}")

In [ ]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_obj,
    eval_dataset=val_dataset_obj,
    data_collator=data_collator,
    tokenizer=tokenizer
)

print("Trainer initialized successfully!")
print(f"Model: {trainer.model.__class__.__name__}")
print(f"Training samples: {len(trainer.train_dataset)}")
print(f"Evaluation samples: {len(trainer.eval_dataset)}")
print(f"Total training steps: {trainer.args.num_train_epochs * len(trainer.train_dataset) // trainer.args.per_device_train_batch_size}")

## 5. Model Training

Start the fine-tuning process with progress monitoring.

In [ ]:
# Start training
print("🚀 Starting model training...")
print(f"Configuration: {NUM_EPOCHS} epochs, batch size {BATCH_SIZE}, learning rate {LEARNING_RATE}")
print(f"Device: {device}")
print(f"Training samples: {len(trainer.train_dataset)}")
print(f"Validation samples: {len(trainer.eval_dataset)}")

# Train the model
training_results = trainer.train()

print("\n✅ Training completed!")
print(f"Final training loss: {training_results.training_loss:.4f}")
print(f"Total training time: {training_results.metrics['train_runtime']:.2f} seconds")
print(f"Training samples per second: {training_results.metrics['train_samples_per_second']:.1f}")

## 6. Training Results Analysis

Analyze training progress and save the final model.

In [ ]:
# Get training logs
training_logs = trainer.state.log_history

# Extract training and validation losses
train_losses = []
eval_losses = []
epochs = []

for log in training_logs:
    if 'train_loss' in log:
        train_losses.append(log['train_loss'])
        epochs.append(log['epoch'])
    if 'eval_loss' in log:
        eval_losses.append(log['eval_loss'])

# Plot training progress
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, 'b-', label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()
plt.grid(True)

if eval_losses:
    eval_epochs = list(range(1, len(eval_losses) + 1))
    plt.subplot(1, 2, 2)
    plt.plot(eval_epochs, eval_losses, 'r-', label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Validation Loss')
    plt.legend()
    plt.grid(True)

plt.tight_layout()
plt.show()

print("=== Training Summary ===")
print(f"Final training loss: {train_losses[-1]:.4f}")
if eval_losses:
    print(f"Final validation loss: {eval_losses[-1]:.4f}")
    print(f"Best validation loss: {min(eval_losses):.4f}")
print(f"Total epochs trained: {len(train_losses)}")

In [ ]:
# Save the final model
final_model_path = "../models/distilbert-squad-finetuned"
os.makedirs(final_model_path, exist_ok=True)

# Save model and tokenizer
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)

# Save training configuration
training_config = {
    "model_name": MODEL_NAME,
    "max_length": MAX_LENGTH,
    "doc_stride": DOC_STRIDE,
    "batch_size": BATCH_SIZE,
    "learning_rate": LEARNING_RATE,
    "num_epochs": NUM_EPOCHS,
    "weight_decay": WEIGHT_DECAY,
    "warmup_steps": WARMUP_STEPS,
    "total_train_steps": TOTAL_TRAIN_STEPS,
    "final_train_loss": training_results.training_loss,
    "training_time_seconds": training_results.metrics['train_runtime'],
    "device": str(device)
}

with open(f"{final_model_path}/training_config.json", "w") as f:
    json.dump(training_config, f, indent=2)

print(f"✅ Model saved to: {final_model_path}")
print(f"✅ Tokenizer saved to: {final_model_path}")
print(f"✅ Training config saved to: {final_model_path}/training_config.json")

# Model size
model_size = sum(p.numel() for p in model.parameters())
print(f"\nModel statistics:")
print(f"  Total parameters: {model_size:,}")
print(f"  Model size: {model_size * 4 / 1024**2:.1f} MB (float32)")

## 7. Model Validation & Testing

Test the trained model on sample questions to verify it's working correctly.

In [ ]:
# Load the trained model for testing
from transformers import pipeline

# Create QA pipeline
qa_pipeline = pipeline(
    "question-answering",
    model=final_model_path,
    tokenizer=final_model_path,
    device=0 if torch.cuda.is_available() else -1
)

print("✅ QA pipeline created successfully!")

# Test on sample examples
test_examples = [
    {
        "context": "The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower. Constructed from 1887 to 1889 as the entrance to the 1889 World's Fair, it was initially criticized by some of France's leading artists and intellectuals for its design, but it has become a global cultural icon of France and one of the most recognizable structures in the world.",
        "question": "Who is the Eiffel Tower named after?"
    },
    {
        "context": "Albert Einstein was a German-born theoretical physicist who developed the theory of relativity, one of the two pillars of modern physics. His work is also known for its influence on the philosophy of science. He is best known to the general public for his mass-energy equivalence formula E = mc², which has been dubbed 'the world's most famous equation'.",
        "question": "What is Einstein most famous for?"
    },
    {
        "context": "The Great Barrier Reef is the world's largest coral reef system composed of over 2,900 individual reefs and 900 islands stretching for over 2,300 kilometers over an area of approximately 344,400 square kilometers. The reef is located in the Coral Sea, off the coast of Queensland, Australia.",
        "question": "Where is the Great Barrier Reef located?"
    }
]

print("\n=== Testing Trained Model ===")
for i, example in enumerate(test_examples, 1):
    result = qa_pipeline(question=example["question"], context=example["context"])
    print(f"\nTest {i}:")
    print(f"Question: {example['question']}")
    print(f"Answer: {result['answer']}")
    print(f"Confidence: {result['score']:.4f}")

## 8. Training Summary & Next Steps

Summarize the training results and outline next steps for Phase V evaluation.

In [ ]:
# Create training summary
training_summary = {
    "phase": "IV - Model Implementation & Fine-Tuning",
    "model": {
        "base_model": MODEL_NAME,
        "fine_tuned_model": final_model_path,
        "parameters": model_size,
        "model_size_mb": model_size * 4 / 1024**2
    },
    "training": {
        "train_samples": len(train_dataset),
        "val_samples": len(val_dataset),
        "train_features": len(train_processed['input_ids']),
        "val_features": len(val_processed['input_ids']),
        "epochs": NUM_EPOCHS,
        "batch_size": BATCH_SIZE,
        "learning_rate": LEARNING_RATE,
        "final_train_loss": training_results.training_loss,
        "training_time_seconds": training_results.metrics['train_runtime']
    },
    "hardware": {
        "device": str(device),
        "mixed_precision": training_args.fp16
    },
    "results": {
        "model_saved": True,
        "pipeline_tested": True,
        "ready_for_evaluation": True
    }
}

# Save summary
with open(f"{final_model_path}/training_summary.json", "w") as f:
    json.dump(training_summary, f, indent=2)

print("=== Phase IV Training Complete ===")
print(f"✅ Model: {MODEL_NAME} fine-tuned on SQuAD v1.1")
print(f"✅ Training samples: {len(train_dataset)}")
print(f"✅ Validation samples: {len(val_dataset)}")
print(f"✅ Epochs: {NUM_EPOCHS}")
print(f"✅ Final loss: {training_results.training_loss:.4f}")
print(f"✅ Model saved: {final_model_path}")
print(f"✅ Training time: {training_results.metrics['train_runtime']:.1f} seconds")
print(f"✅ Device: {device}")

print(f"\n📊 Model ready for Phase V: Evaluation & Error Analysis")
print(f"📁 Model files: {final_model_path}")
print(f"📁 Training logs: {training_args.output_dir}")
print(f"📁 Configuration: {final_model_path}/training_config.json")